In [1]:
import os, time

In [2]:
%run ../utilities/curl_utilities.ipynb
%run ../utilities/postgres_utilities.ipynb

In [3]:
model_name = "GritLM/GritLM-7B"
number_data_rows = 1000

In [4]:
connect_db()

In [5]:
batch_size = 256

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("feature-extraction", model=model_name, trust_remote_code=True, torch_dtype="auto", 
                low_cpu_mem_usage=True, batch_size=batch_size)

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.25it/s]
Device set to use cuda:0


In [7]:
sql = f"select description from film limit {number_data_rows}"
results = execute_sql_results(sql)

In [8]:
test_description = "Starting batch embedding test using local model"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Embeddings computed ...", end=" ")
descriptions = list()
for index, film in enumerate(results):
    descriptions.append(film[0])

start_offset = 0
end_offset = batch_size
embeddings = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    embeddings = embeddings + pipe(descriptions[start_offset:end_offset])
    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

assert len(embeddings) == index+1
for e in embeddings[0][0]:
    assert(len(e) == 4096)

toc = time.perf_counter()
print("")
print(f"Time to compute {number_data_rows} embeddings: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch embedding test using local model
-----------------------------------------------
Embeddings computed ... 0 256 256 512 512 768 768 1000 
Time to compute 1000 embeddings: 16.2847 seconds
-----------------------------------------------


In [9]:
close_db()